In [2]:
from torch.utils.mobile_optimizer import optimize_for_mobile
from pytorch_grad_cam.utils.image import show_cam_on_image
from tqdm import tqdm
import numpy as np
import torch.nn.functional as F
import cv2
import torch
from torchvision import transforms
import urllib
from PIL import Image

from model.segmentation.segmentation_output_wrapper import SegmentationModelOutputWrapper



sem_classes = ['__background__', 'cable', 'tower_lattice', 'tower_tucohy', 'tower_wooden']
model = torch.hub.load('pytorch/vision:v0.11.0', 'deeplabv3_resnet101', pretrained=False,
                       num_classes=len(sem_classes))

PATH = f'model/segmentation/model_ResNet101.pth'

if torch.cuda.is_available():
    model.load_state_dict(torch.load(PATH))
else:
    model.load_state_dict(torch.load(PATH, map_location=torch.device('cpu')))


model = SegmentationModelOutputWrapper(model)

filename = 'data/images/3_00092.jpg'

input_image = Image.open(filename)
input_image = input_image.convert("RGB")
preprocess = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0)  # create a mini-batch as expected by the model

# Convert to ONNX
torch.onnx.export(model, input_batch, "model/segmentation/model_ResNet101.onnx", opset_version=11)

Using cache found in /Users/thanhhung/.cache/torch/hub/pytorch_vision_v0.11.0


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================
